In [1]:
cd ../../

/Users/in-divye.singh/Documents/Projects/MIC_predictor


In [2]:
from notebooks.utils import *

In [3]:
import biovec
import numpy as np
import pandas as pd
from itertools import chain

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, ShuffleSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

### Physico-chemical properties

In [6]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [7]:
params = ['instability_index', 'isoelectric_point', 'helix', 'turn', 'sheet', 'with_reduced_cysteines',
          'with_disulfid_bridges', 'net_charge_at_pH7point4']

In [8]:
def get_all_prop(df):
    prop = []
    for seq in df.Sequence:
        X = ProteinAnalysis(seq)
#         molecular_weight = X.molecular_weight()
#         aromaticity = X.aromaticity()
        instability_index = X.instability_index()
        isoelectric_point = X.isoelectric_point()
        sec_struc = X.secondary_structure_fraction()
        helix = sec_struc[0]
        turn = sec_struc[1]
        sheet = sec_struc[2]
        epsilon_prot = X.molar_extinction_coefficient()
        with_reduced_cysteines = epsilon_prot[0]
        with_disulfid_bridges = epsilon_prot[1]
#         gravy = X.gravy() # hydrophobicity related
        # flexibility = X.flexibility()
        # X.protein_scale()
        net_charge_at_pH7point4 = X.charge_at_pH(7.4)

        prop.append([instability_index, isoelectric_point, helix, turn, sheet,
                     with_reduced_cysteines, with_disulfid_bridges, net_charge_at_pH7point4])
    return pd.concat([df, pd.DataFrame(prop, columns=params)], axis=1)

In [9]:
feat_105 = pd.read_csv("data/raw/105_feature_ha_avp_ic50.csv")

In [84]:
feat_105

,Sequence,MIC,hpi_<-1.5_frac,hpi_<-2.0_frac,hpi_<-2.5_frac,hpi_<-1.5,hpi_<-2.0,hpi_<-2.5,fraction_A,fraction_C,...,T_lcfraction,V_lcscore,V_lcfraction,W_lcscore,W_lcfraction,Y_lcscore,Y_lcfraction,lcs_fractions,lcs_scores,lcs_lowest_complexity
0,AAQRRGRVGRNPNQVGD,442.00000,0.117647,0.000000,0.0,2,0,0,0.117647,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
1,HRILARIRQMMT,435.50000,0.000000,0.000000,0.0,0,0,0,0.083333,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
2,RNPSQVGD,383.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
3,RVGRNPNQVGD,374.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,11,7
4,AAQRRGRIGRNPSQVGD,358.00000,0.000000,0.000000,0.0,0,0,0,0.117647,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,NGAICWGPCPTAFRQIGNCGHFKVRCCKIR,0.11515,0.000000,0.000000,0.0,0,0,0,0.066667,0.166667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,11
708,CFPYITRPGTYHDWWYTRKNRQ,0.30000,0.500000,0.045455,0.0,11,1,0,0.000000,0.045455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,12
709,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,0.01131,0.305556,0.138889,0.0,11,5,0,0.027778,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055556,2,7
710,GTNLSVPNPLGFFPDHQLDPAFGANSNNPDWDFNPNKDHWPEANKVG,0.00008,0.297872,0.127660,0.0,14,6,0,0.063830,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,9


In [10]:
def print_duplicate_col(df):
    for col in df.columns:
        if len(df[col].shape) > 1:
            print(col, df[col].shape)

In [11]:
print_duplicate_col(feat_105)

In [12]:
feat_105

,Sequence,MIC,hpi_<-1.5_frac,hpi_<-2.0_frac,hpi_<-2.5_frac,hpi_<-1.5,hpi_<-2.0,hpi_<-2.5,fraction_A,fraction_C,...,T_lcfraction,V_lcscore,V_lcfraction,W_lcscore,W_lcfraction,Y_lcscore,Y_lcfraction,lcs_fractions,lcs_scores,lcs_lowest_complexity
0,AAQRRGRVGRNPNQVGD,442.00000,0.117647,0.000000,0.0,2,0,0,0.117647,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
1,HRILARIRQMMT,435.50000,0.000000,0.000000,0.0,0,0,0,0.083333,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
2,RNPSQVGD,383.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,8
3,RVGRNPNQVGD,374.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,11,7
4,AAQRRGRIGRNPSQVGD,358.00000,0.000000,0.000000,0.0,0,0,0,0.117647,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,NGAICWGPCPTAFRQIGNCGHFKVRCCKIR,0.11515,0.000000,0.000000,0.0,0,0,0,0.066667,0.166667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,11
708,CFPYITRPGTYHDWWYTRKNRQ,0.30000,0.500000,0.045455,0.0,11,1,0,0.000000,0.045455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,12
709,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,0.01131,0.305556,0.138889,0.0,11,5,0,0.027778,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055556,2,7
710,GTNLSVPNPLGFFPDHQLDPAFGANSNNPDWDFNPNKDHWPEANKVG,0.00008,0.297872,0.127660,0.0,14,6,0,0.063830,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,9


In [13]:
df = get_all_prop(feat_105)

In [15]:
aa_freq = reduce_by_kmer_frequency(df)
aa_freq.columns = list(map(lambda s: 'mono_'+s, aa_freq.columns))

############# Dipep freq #############
dipep_freq = reduce_by_kmer_frequency(df, kmer=2)
dipep_freq.columns = list(map(lambda s: 'di_'+s, dipep_freq.columns))

############# Tripep freq #############
tripep_freq = reduce_by_kmer_frequency(df, kmer=3)

############# Prot2Vec #############
uniprot_embedding = biovec.models.load_protvec("data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df['Sequence'], uniprot_embedding, kmer=3)
avg_protvec = avg_protvec.reset_index(drop=True)
avg_protvec.columns = list(map(lambda s: 'vec_'+str(s), avg_protvec.columns))

/Users/in-divye.singh/anaconda3/envs/life-science/lib/python3.6/site-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Creating vectors: 100%|██████████| 712/712 [00:00<00:00, 1158.22sequence/s]


In [21]:
df = pd.concat([df, aa_freq, dipep_freq, avg_protvec], axis=1)
df

,Sequence,MIC,hpi_<-1.5_frac,hpi_<-2.0_frac,hpi_<-2.5_frac,hpi_<-1.5,hpi_<-2.0,hpi_<-2.5,fraction_A,fraction_C,...,vec_90,vec_91,vec_92,vec_93,vec_94,vec_95,vec_96,vec_97,vec_98,vec_99
0,AAQRRGRVGRNPNQVGD,442.00000,0.117647,0.000000,0.0,2,0,0,0.117647,0.000000,...,0.015608,0.000107,0.001647,0.035378,-0.003124,-0.021026,-0.034534,0.032184,-0.022649,0.047560
1,HRILARIRQMMT,435.50000,0.000000,0.000000,0.0,0,0,0,0.083333,0.000000,...,0.096292,-0.031165,0.026491,0.094631,-0.026952,-0.054827,0.009290,-0.011064,0.026011,0.021778
2,RNPSQVGD,383.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,0.069664,-0.045011,-0.009747,-0.030202,0.029767,0.038564,-0.049012,-0.021545,-0.000864,0.000764
3,RVGRNPNQVGD,374.00000,0.000000,0.000000,0.0,0,0,0,0.000000,0.000000,...,0.008095,-0.045382,0.012418,-0.000806,-0.028781,-0.008842,-0.021954,0.001664,-0.028276,0.020661
4,AAQRRGRIGRNPSQVGD,358.00000,0.000000,0.000000,0.0,0,0,0,0.117647,0.000000,...,0.040302,-0.001207,0.012745,0.021320,0.014074,0.006161,-0.029614,-0.004915,0.007278,0.045714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,NGAICWGPCPTAFRQIGNCGHFKVRCCKIR,0.11515,0.000000,0.000000,0.0,0,0,0,0.066667,0.166667,...,0.006357,0.000806,0.007335,-0.015952,0.054503,0.056923,-0.067147,0.017195,0.037202,-0.061830
708,CFPYITRPGTYHDWWYTRKNRQ,0.30000,0.500000,0.045455,0.0,11,1,0,0.000000,0.045455,...,0.020611,0.010793,0.019357,0.026522,0.038127,0.043636,0.014737,0.010608,0.054837,-0.038420
709,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,0.01131,0.305556,0.138889,0.0,11,5,0,0.027778,0.000000,...,0.051252,-0.057756,0.006149,0.001708,0.016342,0.048390,-0.021271,-0.017798,0.070233,-0.046075
710,GTNLSVPNPLGFFPDHQLDPAFGANSNNPDWDFNPNKDHWPEANKVG,0.00008,0.297872,0.127660,0.0,14,6,0,0.063830,0.000000,...,0.023278,-0.013674,-0.007323,-0.014028,-0.024394,0.045786,-0.018962,0.007527,0.048972,-0.061540


In [23]:
X = df.drop(["Sequence", "MIC"], axis=1)

In [24]:
nan_col = []
for col in X.columns:
    if np.any(np.isnan(X[col].values)):
        nan_col.append(col)

In [25]:
X = X.drop(nan_col, axis=1)

In [26]:
y = df["MIC"]

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [80]:
def cross_validation(model, X, y, metric):
    score = make_scorer(metric)
    kf = ShuffleSplit(n_splits=5, random_state=42)
    result = cross_val_score(model, X = X, y = y, scoring = score, cv = kf)
    return result

In [35]:
clf = KNeighborsRegressor(n_neighbors=1)
_ = clf.fit(X_train, y_train)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
perm_importance = permutation_importance(clf, X_train, y_train, scoring=mape_scorer,
                                         n_repeats=10,n_jobs=-1,random_state=42)
sorted_idx = perm_importance.importances_mean.argsort()[::-1]

In [36]:
len(X_train.columns)

590

In [37]:
list(zip(X_train.columns[sorted_idx], perm_importance.importances_mean[sorted_idx]))

[('with_reduced_cysteines', 787021.4514081978),
 ('with_disulfid_bridges', 584727.618545837),
 ('molecular_weight', 390484.3967832803),
 ('instability_index', 30753.20131234076),
 ('lcs_scores', 22.92975221495382),
 ('net_charge_at_pH7point4', 16.182207825835274),
 ('hpi_<-1.5', 13.503559534203252),
 ('isoelectric_point', 1.5396688477760985),
 ('IEP', 1.5396688477760985),
 ('lcs_lowest_complexity', 0.9192208954700082),
 ('hpi_<-2.0', 0.27865371041431886),
 ('gravy', 0.21552337120839668),
 ('sheet', 0.03893470325807759),
 ('beta_sheet', 0.03893470325807759),
 ('length', 0.016857644991212655),
 ('mono_I', 0.009733675814519397),
 ('fraction_I', 0.009733675814519397),
 ('di_GV', 0.0),
 ('di_GT', 0.0),
 ('di_GS', 0.0),
 ('di_GR', 0.0),
 ('di_GQ', 0.0),
 ('di_GP', 0.0),
 ('di_GN', 0.0),
 ('di_GM', 0.0),
 ('di_GL', 0.0),
 ('di_GK', 0.0),
 ('di_GI', 0.0),
 ('di_GY', 0.0),
 ('di_GW', 0.0),
 ('di_HI', 0.0),
 ('di_HQ', 0.0),
 ('di_HP', 0.0),
 ('di_HN', 0.0),
 ('di_HM', 0.0),
 ('di_HL', 0.0),
 ('d

In [38]:
score = []
for i in range(1,100):#np.r_[1:25, np.arange(25,10,5)]:
    print(sorted_idx[:i])    
    cv_r2 = cross_validation(model = clf,
                        X = X_train.iloc[:,sorted_idx[:i]],
                        y = y_train,
                        metric = mean_absolute_percentage_error)
#     clf = SVR()
    _ = clf.fit(X_train.iloc[:,sorted_idx[:i]],y_train)
    y_pred = clf.predict(X_test.iloc[:,sorted_idx[:i]])
    test_r2 = mean_absolute_percentage_error(y_test,y_pred)
    score.append([sorted_idx[:i],np.mean(cv_r2), test_r2])

score_df = pd.DataFrame(score,columns = ['features','cv_mape','test_mape'])

[67]
[67 68]
[67 68 28]
[67 68 28 62]
[67 68 28 62 60]
[67 68 28 62 60 69]
[67 68 28 62 60 69  3]
[67 68 28 62 60 69  3 63]
[67 68 28 62 60 69  3 63 27]
[67 68 28 62 60 69  3 63 27 61]
[67 68 28 62 60 69  3 63 27 61  4]
[67 68 28 62 60 69  3 63 27 61  4 29]
[67 68 28 62 60 69  3 63 27 61  4 29 66]
[67 68 28 62 60 69  3 63 27 61  4 29 66 57]
[67 68 28 62 60 69  3 63 27 61  4 29 66 57 26]
[67 68 28 62 60 69  3 63 27 61  4 29 66 57 26 77]
[67 68 28 62 60 69  3 63 27 61  4 29 66 57 26 77 13]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202

[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171 170 169 168
 167 166 164 174 163 162 161 160 159 158 157 173 175]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171 170 169 168
 167 166 164 174 163 162 161 160 159 158 157 173 175 193]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171 170 169 168
 167 166 164 174 163 162 161 160 159 158 157 173 175 193 185]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 21

[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171 170 169 168
 167 166 164 174 163 162 161 160 159 158 157 173 175 193 185 192 225 190
 189 188 187 186 184 176 183 182 181 180 179 178 177 224 589 226 269 275
 274 273 272 271 270 268 260]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171 170 169 168
 167 166 164 174 163 162 161 160 159 158 157 173 175 193 185 192 225 190
 189 188 187 186 184 176 183 182 181 180 179 178 177 224 589 226 269 275
 274 273 272 271 270 268 260 267]
[ 67  68  28  62  60  69   3  63  27  61   4  29  66  57  26  77  13 207
 206 205 204 203 202 201 200 199 198 197 209 208 217 223 222 221 220 219
 218 216 195 215 214 213 212 211 210 196 191 194 165 172 171

In [39]:
score_df.sort_values(by='test_mape')

,features,cv_mape,test_mape
2,"[67, 68, 28]",119964.103962,1.239381e+05
0,[67],385566.143455,5.676959e+06
81,"[67, 68, 28, 62, 60, 69, 3, 63, 27, 61, 4, 29,...",116850.520432,6.324518e+06
82,"[67, 68, 28, 62, 60, 69, 3, 63, 27, 61, 4, 29,...",87794.340904,6.324518e+06
83,"[67, 68, 28, 62, 60, 69, 3, 63, 27, 61, 4, 29,...",79481.293963,6.324518e+06
...,...,...,...
6,"[67, 68, 28, 62, 60, 69, 3]",117011.671684,6.324533e+06
3,"[67, 68, 28, 62]",77179.848991,6.324538e+06
5,"[67, 68, 28, 62, 60, 69]",45308.626732,6.324540e+06
4,"[67, 68, 28, 62, 60]",192327.427689,6.324551e+06


In [40]:
bbb = list(df.columns[132:532])+list(df.columns[-100:])

In [31]:
len(bbb)

500

In [34]:
clf = KNeighborsRegressor(n_neighbors=1, weights='uniform', metric='euclidean')
_ = clf.fit(X_train[bbb],y_train)
y_pred = clf.predict(X_test[bbb])
print(mean_absolute_percentage_error(y_test,y_pred), r2_score(y_test,y_pred))

38427.619517675776 0.18490034906192654


In [41]:
clf = KNeighborsRegressor(n_neighbors=1)
_ = clf.fit(X_train[bbb], y_train)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
perm_importance = permutation_importance(clf, X_train[bbb], y_train, scoring=mape_scorer,
                                         n_repeats=10,n_jobs=-1,random_state=42)
sorted_idx = perm_importance.importances_mean.argsort()[::-1]

In [42]:
len(X_train[bbb].columns)

500

In [43]:
list(zip(X_train[bbb].columns[sorted_idx], perm_importance.importances_mean[sorted_idx]))

[('di_SI', 22.95212653778559),
 ('vec_91', 10.404217926186293),
 ('vec_57', 4.462248871429064),
 ('vec_40', 2.3374340949033394),
 ('vec_34', 2.2719835161505366),
 ('di_LI', 1.404125443808407),
 ('di_RS', 1.2787103811890188),
 ('di_LN', 0.8787346221441126),
 ('di_EL', 0.6449912126537786),
 ('di_KL', 0.4710017574692443),
 ('di_EN', 0.36764020940587605),
 ('di_GS', 0.3581677826967141),
 ('di_SD', 0.29374843083103197),
 ('di_GA', 0.29291154071470415),
 ('di_LE', 0.17031474676465888),
 ('di_TD', 0.14730367218573676),
 ('di_RL', 0.12302284710017575),
 ('di_NI', 0.1183999921563437),
 ('di_LL', 0.11363650162215508),
 ('di_TW', 0.07908611599297014),
 ('di_DS', 0.07029876977152899),
 ('di_KV', 0.0676153443691524),
 ('vec_62', 0.059975293734085874),
 ('di_AL', 0.05858230814294084),
 ('di_LH', 0.048330404217926184),
 ('di_KA', 0.035149384885764495),
 ('vec_76', 0.03263871453678133),
 ('di_LR', 0.03195398625978591),
 ('di_IL', 0.031321234056621836),
 ('di_AA', 0.030218085804437043),
 ('di_IK', 0.01

In [81]:
score = []
for i in range(1,X_train[bbb].shape[1]):#np.r_[1:25, np.arange(25,10,5)]:
    print(sorted_idx[:i])    
    cv_r2 = cross_validation(model = clf,
                        X = X_train[bbb].iloc[:,sorted_idx[:i]],
                        y = y_train,
                        metric = mean_absolute_percentage_error)
#     clf = SVR()
    _ = clf.fit(X_train[bbb].iloc[:,sorted_idx[:i]],y_train)
    y_pred = clf.predict(X_test[bbb].iloc[:,sorted_idx[:i]])
    test_mape = mean_absolute_percentage_error(y_test,y_pred)
    test_r2 = r2_score(y_test,y_pred)
    score.append([sorted_idx[:i],np.mean(cv_r2), test_mape, test_r2])

score_df = pd.DataFrame(score,columns = ['features','cv_mape','test_mape', 'test_r2'])

[307]
[307 491]
[307 491 457]
[307 491 457 440]
[307 491 457 440 434]
[307 491 457 440 434 187]
[307 491 457 440 434 187 295]
[307 491 457 440 434 187 295 191]
[307 491 457 440 434 187 295 191  69]
[307 491 457 440 434 187 295 191  69 169]
[307 491 457 440 434 187 295 191  69 169  71]
[307 491 457 440 434 187 295 191  69 169  71 115]
[307 491 457 440 434 187 295 191  69 169  71 115 302]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55]
[307 491 457 440 434 187 295 19

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 41

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 15

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219]
[307 491 457 440 434 187 295 191  69 169  71 115 302

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484]
[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 

...

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484 178 179 180 181 182 483 184 481 480 188
 479 190 478 192 193 477 195 196 197 116 113 114  28  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  29  27  56  26   1   2   3   4   5   6   7   8 497  10  11  12
  13  14  15  16  17  18  19  20  21  22 496  24  25 495  57 228  86  88
 492  90  91  92  93  94  95  96  97  98  99 490 101 102 103 104 105 106
 107 108 109 110 111 112  87  85  58  84  59  60  6

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484 178 179 180 181 182 483 184 481 480 188
 479 190 478 192 193 477 195 196 197 116 113 114  28  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  29  27  56  26   1   2   3   4   5   6   7   8 497  10  11  12
  13  14  15  16  17  18  19  20  21  22 496  24  25 495  57 228  86  88
 492  90  91  92  93  94  95  96  97  98  99 490 101 102 103 104 105 106
 107 108 109 110 111 112  87  85  58  84  59  60  6

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484 178 179 180 181 182 483 184 481 480 188
 479 190 478 192 193 477 195 196 197 116 113 114  28  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  29  27  56  26   1   2   3   4   5   6   7   8 497  10  11  12
  13  14  15  16  17  18  19  20  21  22 496  24  25 495  57 228  86  88
 492  90  91  92  93  94  95  96  97  98  99 490 101 102 103 104 105 106
 107 108 109 110 111 112  87  85  58  84  59  60  6

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484 178 179 180 181 182 483 184 481 480 188
 479 190 478 192 193 477 195 196 197 116 113 114  28  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  29  27  56  26   1   2   3   4   5   6   7   8 497  10  11  12
  13  14  15  16  17  18  19  20  21  22 496  24  25 495  57 228  86  88
 492  90  91  92  93  94  95  96  97  98  99 490 101 102 103 104 105 106
 107 108 109 110 111 112  87  85  58  84  59  60  6

[307 491 457 440 434 187 295 191  69 169  71 115 302 100 183 322 289 227
 189 338  55 177 462   9 186 160 476 194 149   0 148  89  23 482 371 499
 329 256 185 382 360 412 146 147 488 487 150 151 152 153 154 155 463 144
 156 157 158 159 486 161 162 163 164 165 166 167 145 142 143 485 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 168 171 170 200 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 201 199
 489 198 172 173 174 175 176 484 178 179 180 181 182 483 184 481 480 188
 479 190 478 192 193 477 195 196 197 116 113 114  28  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  29  27  56  26   1   2   3   4   5   6   7   8 497  10  11  12
  13  14  15  16  17  18  19  20  21  22 496  24  25 495  57 228  86  88
 492  90  91  92  93  94  95  96  97  98  99 490 101 102 103 104 105 106
 107 108 109 110 111 112  87  85  58  84  59  60  6

In [83]:
score_df.sort_values(by='test_mape')

,features,cv_mape,test_mape,test_r2
2,"[307, 491, 457]",40195.035037,2.962532e+04,-0.722658
496,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",7457.905250,3.841634e+04,0.185945
497,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",7458.538373,3.841783e+04,0.184900
498,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",7447.693601,3.842762e+04,0.184900
149,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",8126.868073,3.930128e+04,0.018512
...,...,...,...,...
33,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",10821.086442,4.966762e+07,-0.265316
32,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",10766.137602,4.966777e+07,-0.652141
30,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",10744.431117,4.966779e+07,-0.839987
31,"[307, 491, 457, 440, 434, 187, 295, 191, 69, 1...",10761.338425,4.966779e+07,-0.748426
